### <strong>Motivação:</strong> o site do SAE (o serviço de assistência social da Unicamp) está sendo migrado para uma nova arquitetura, então me solicitaram que eu extraisse a base e para enviar ao time de devs tercerizados.<br><br> Este trabalho faz parte de uma bolsa de assistência social que eu tenho

### <strong>Tecnologia:</strong> Foi optado de fazer uma scraping via html porque não foi encontrada nanhuma api aberta ou xml page, por isso o uso do BeautifulSoup

## Importação de libs

In [60]:
from bs4 import BeautifulSoup
import requests
import json

### Função de extração de dados de posts de uma page

In [71]:
def extract_post(posts):
    results = []
    for sample in posts:

        ### post desceription data
        published_date_soup = sample.find(class_ = 'tm-article-date')

        published_date = published_date_soup.find(class_ = 'tm-article-date-day').get_text().strip() + ' ' + published_date_soup.find(class_ = 'tm-article-date-year').get_text().strip()

        article_title = sample.find(class_ = 'uk-article-title').get_text().strip()

        ### paragraphs extract
        paragraphs_soup = sample.find_all('p')

        paragraphs_emphasis = [p.get_text().strip() for p in paragraphs_soup if p.get_text().strip() != '' and p.find('strong')]
        
        paragraphs = [p.get_text().strip() for p in paragraphs_soup if p.get_text().strip() != '' and not p.find('strong')]

        results.append({
            'title': article_title,
            'publishedDate': published_date,
            'paragraphs': paragraphs,
            'emphasisParagraphs': paragraphs_emphasis
        })

    return results
    

### Função de runtime (estou fazendo um loop para extrair um database com um range grande)

In [72]:
def main():
    counter = 0

    headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53','Accept-Language': 'en-US,en;q=0.9,it;q=0.8,es;q=0.7'
    }
    results_pages = []

    ## loop for extracting data from multiple pages
    for i in range(1,100):
        ## url formatation
        print(f'Extraindo dados da page {i}')
        url = ''
        if counter == 0:
            url = f'https://www.sae.unicamp.br/portal/pt/'
        else:
            url = f'https://www.sae.unicamp.br/portal/pt/?start={counter}'
        
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.text)
        
        posts = soup.find_all(class_ = 'uk-article tm-article')
        
        ## data extract
        results = extract_post(posts)

        results_pages.append(results)
        
        counter += 3
    
    with open('posts.json', 'w') as file:
        json.dump(results_pages, file, ensure_ascii=False)

    return {
        'statusCode': 200,
        'message': 'Extraction success!'
    }
